In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import statistics 
from scipy.integrate import quad
def distribution_function(data):
    actual_data=0
    F=[]
    for i in range(0,len(data)):
        actual_data+=data[i]/100
        F.append(actual_data)
    return(F)

In [ ]:
data=[5,8,6,12,14,18,11,6,13,7]
F_empirical=distribution_function(data)
N=sum(data)
print(F_empirical)

[0.05, 0.13, 0.19, 0.31, 0.45, 0.63, 0.74, 0.8, 0.93, 1.0]


Пункт b)

Найдём параметры нормального распределения максимизирую функцию максимального правдоподобия

In [ ]:
limits = np.array(-np.inf)
limits = np.append(limits, np.arange(0.5, 9, 1))
limits = np.append(limits, np.inf)
def normal_dencity(x, teta_1, teta_2):
    return 1/(np.sqrt(2*np.pi)*teta_2)*np.exp(-(x-teta_1)**2/(2*teta_2**2))
maxim = 0 
best_teta_1 = 0
best_teta_2 = 0
for teta_1 in np.arange(1, 10, 0.1):
    for teta_2 in np.arange(1, 10, 0.1):
        L = 1
        for i in range(0, len(limits)-1, ):
                L *= quad(normal_dencity, limits[i], limits[i+1], args = (teta_1, teta_2))[0]**data[i]
        if (L > maxim):
                maxim, best_teta_1, best_teta_2 = L, teta_1, teta_2
print(best_teta_1, best_teta_2)

4.800000000000003 2.7000000000000015


Хи-квадрат

In [ ]:
probabilities = np.array([quad(normal_dencity, limits[i], limits[i+1], args = (best_teta_1, best_teta_2))[0] for i in range(0, len(limits)-1)])

In [ ]:
delta_hi = 0
for i in range(len(data)):
    delta_hi += (data[i]-N*probabilities[i])**2/(N*probabilities[i])
print(delta_hi)

9.779231990914932


Колмогоров

In [ ]:
arr_f = np.array([])
for i in range(0,10):
    arr_f =  np.append(arr_f, quad(normal_dencity, -np.inf, i, args = (best_teta_1, best_teta_2))[0])
delta_b_kol = max(abs(arr_f - F_empirical))*np.sqrt(N)
print( "delta  =", delta_b_kol)

delta  = 1.0047572223260337


Параметрический Бутстрап:

In [12]:
delta_boot = np.array([])

for i in range(0, 10000):
    
    bootstrap = np.sort(np.random.normal(best_teta_1, best_teta_2, N).astype(int))
    count = np.array([])
    for j in [0,1,2,3,4,5,6,7,8,9]: 
        count = np.append(count, sum(bootstrap == j)) 
        
    delta_i_Mom = 0
    teta_1_boot = np.mean(bootstrap)
    teta_2_boot = (np.mean(bootstrap**2) - teta_1_boot**2)**0.5
    probabilities_Mom_i = np.array([quad(normal_dencity, limits[k], limits[k+1], args = (teta_1_boot, teta_2_boot))[0] for k in range(0, len(limits)-1)])
    for j in range(0,10):
        delta_i_Mom +=  (count[j] - probabilities_Mom_i[j]*100)**2/(100*probabilities_Mom_i[j])
    delta_boot = np.append(delta_boot, delta_i_Mom)
    delta_boot.sort()

In [13]:
delta_boot.sort() 
k_ = sum(delta_boot < delta_b_kol)
k_ += 1
p_value_b_kol = 1 - (k_ - 1)/10000
print("p_value in Kolmogorov=", p_value_b_kol)

p_value in Kolmogorov= 0.9997


Пункт С) Метод моментов

In [14]:
samples = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
                 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7,
                 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9])
M_1 = np.mean(samples)
M_2 = np.mean(samples**2) 
teta_1_Mom = M_1
teta_2_Mom = (M_2 - M_1**2)**0.5
print('teta_1 in method of moments:',teta_1_Mom,'teta_2 in method of moments:',teta_2_Mom)
probabilities_Mom = np.array([quad(normal_dencity, limits[i], limits[i+1], args = (teta_1_Mom, teta_2_Mom))[0] for i in range(0, len(limits)-1)])

teta_1 in method of moments: 4.77 teta_2 in method of moments: 2.5054141374231933


In [15]:
delta_Mom = 0
for i in range(len(data)):
    delta_Mom += (data[i] - probabilities_Mom[i]*100)**2/(100*probabilities_Mom[i])
print('delta in method of moments',delta_Mom)

delta in method of moments 10.798973140500914


In [16]:
delta_Mom_Boot = np.array([])

for i in range(0, 10000):
    
    bootstrap = np.sort(np.random.normal(teta_1_Mom, teta_2_Mom, N).astype(int))
    count = np.array([])
    for j in [0,1,2,3,4,5,6,7,8,9]: 
        count = np.append(count, sum(bootstrap == j)) 
        
    delta_i_Mom = 0
    teta_1_Mom_boot = np.mean(bootstrap)
    teta_2_Mom_boot = (np.mean(bootstrap**2) - teta_1_Mom_boot**2)**0.5
    probabilities_Mom_i = np.array([quad(normal_dencity, limits[k], limits[k+1], args = (teta_1_Mom_boot, teta_2_Mom_boot))[0] for k in range(0, len(limits)-1)])
    for j in range(0,10):
        delta_i_Mom +=  (count[j] - probabilities_Mom_i[j]*100)**2/(100*probabilities_Mom_i[j])
    delta_Mom_Boot = np.append(delta_Mom_Boot, delta_i_Mom)
    delta_Mom_Boot.sort()

In [17]:
k_Mom = sum(delta_Mom_Boot < delta_Mom)
k_Mom += 1
p_value_Mom = 1 - (k_Mom - 1)/10000
print("p_value in method of moments =",p_value_Mom)

p_value in method of moments = 0.20879999999999999
